In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import chromedriver_autoinstaller
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm
import requests

In [2]:
url = 'https://map.naver.com/v5/search'
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)  # 드라이버 경로
driver.get(url)
key_word = '마포구 치킨'

C:\Users\Heejun\AppData\Local\Temp/ipykernel_3840/2593937393.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)  # 드라이버 경로


In [3]:
# 검색창 찾기
sleep(1)
search = driver.find_element_by_css_selector('div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'lxml')  # html 파싱하여  가져온다


C:\Users\Heejun\AppData\Local\Temp/ipykernel_3840/3552456074.py:3: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  search = driver.find_element_by_css_selector('div.input_box > input.input_search')


In [4]:
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    res
    soup
def page_down(num):
    body = driver.find_element_by_css_selector('body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [5]:
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [6]:
switch_frame('searchIframe')
page_down(40)

C:\Users\Heejun\AppData\Local\Temp/ipykernel_3840/2706137607.py:7: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body = driver.find_element_by_css_selector('body')


In [7]:
# 매장 리스트
store_list = driver.find_elements_by_css_selector('._1EKsQ')
# 페이지 리스트
next_btn = driver.find_elements_by_css_selector('._2ky45 > a')

store_dict = {'매장정보': []}
start = time.time()
print('[크롤링 시작...]')

[크롤링 시작...]


C:\Users\Heejun\AppData\Local\Temp/ipykernel_3840/813670217.py:2: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  store_list = driver.find_elements_by_css_selector('._1EKsQ')
C:\Users\Heejun\AppData\Local\Temp/ipykernel_3840/813670217.py:4: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  next_btn = driver.find_elements_by_css_selector('._2ky45 > a')


In [8]:
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    store_list
    for data in range(len(store_list)):  # 매장 리스트 만큼
        try:
            page = driver.find_elements_by_css_selector('.OXiLu')
            page[data].click()
            sleep(1)
        except:
            pass
        try:
            # 상세 페이지로 이동
            switch_frame('entryIframe')
            time_wait(1, '._3XamX')
            # 스크롤을 맨밑으로 1초간격으로 내린다.
            for down in range(3):
                sleep(1)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            store_name = driver.find_element_by_css_selector('._3XamX').text
            store_url = driver.find_elements_by_css_selector('._1Y6hi')
            store_url = store_url[1].find_element_by_tag_name('a').get_attribute('href')
            
            dict_temp = {
                'name': store_name,
                'url': store_url
            }
            store_dict['매장정보'].append(dict_temp)
            print(f'{store_name} ...완료')
            switch_frame('searchIframe')
            sleep(1)

        except:
            print('ERROR!' * 3)
            switch_frame('searchIframe')
            sleep(1)
    if page[-1]:  # 마지막 매장일 경우 다음버튼 클릭
        next_btn[-1].click()
        sleep(2)
    else:
        print('페이지 인식 못함')
        break
        
print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을닫는다.

C:\Users\Heejun\AppData\Local\Temp/ipykernel_3840/2641432176.py:5: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  page = driver.find_elements_by_css_selector('.OXiLu')
C:\Users\Heejun\AppData\Local\Temp/ipykernel_3840/2641432176.py:18: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  store_name = driver.find_element_by_css_selector('._3XamX').text
C:\Users\Heejun\AppData\Local\Temp/ipykernel_3840/2641432176.py:19: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  store_url = driver.find_elements_by_css_selector('._1Y6hi')
D:\anaconda\lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.w

홍대씨부엉 ...완료
아웃닭 홍대점 ...완료
철인7호 치킨 홍대점 ...완료
치킨인더키친 ...완료
노랑통닭 홍대점 ...완료
누구나홀딱반한닭 홍대점 ...완료
치킨앤카레군 ...완료
또바기치킨 상수홍대점 ...완료
큐스닭강정 ...완료
닭날다 ...완료
레게치킨 ...완료
치킨친구 ...완료
더블플레이 치킨 동교동본점 ...완료
닭꼬얌 ...완료
깐부치킨 합정메세나폴리스점 ...완료
오늘통닭 도화점 ...완료
네네치킨 서교점 ...완료
계림원 공덕역점 ...완료
바른치킨 공덕파크자이점 ...완료
BHC치킨 홍대점 ...완료
교촌치킨 홍대점 ...완료
교촌치킨 상암점 ...완료
노랑통닭 서울상암점 ...완료
컬투치킨 합정사랑점 ...완료
교촌치킨 도화점 ...완료
푸라닭 망원점 ...완료
호치킨 용강점 ...완료
꼬꼬순이 동교본점 ...완료
깐부치킨 홍대점 ...완료
로켓치킨 ...완료
맘스터치 홍대점 ...완료
교촌치킨 신공덕점 ...완료
노랑통닭 망원점 ...완료
네네치킨 마포역점 ...완료
치킨플러스 성산점 ...완료
누구나홀딱반한닭 망원역점 ...완료
호치킨 상암9단지점 ...완료
BHC치킨 상암DMC점 ...완료
굽네치킨 연남점 ...완료
치킨버스 상암점 ...완료
교촌치킨 연남점 ...완료
BHC치킨 공덕점 ...완료
치킨처럼 아현점 ...완료
또바기치킨 서교점 ...완료
네오치킨 ...완료
페리카나 성산동점 ...완료
교촌치킨 성산2동점 ...완료
네네치킨 망원점 ...완료
BBQ치킨 공덕점 ...완료
BBQ서교점 ...완료
페리카나 공덕역점 ...완료
BHC치킨 합정역점 ...완료
깐부치킨 공덕오거리점 ...완료
계림원 망원점 ...완료
노랑통닭 상수점 ...완료
에그랑누룽지통닭구이 ...완료
매드후라이공덕점 ...완료
처갓집양념치킨 도화점 ...완료
치킨천국 ...완료
쌀통에 퐁닭 ...완료
본스치킨대흥점 ...완료
교촌치킨 망원2동점 ...완료
깐부치킨 홍대동교동점 ...완료
교동닭강정 ...완료
굽네치킨 망원1동점 ...완료
노랑통닭 아현

In [9]:
import pandas as pd
store_list = []
for i in store_dict['매장정보']:
    store_list.append(i)
store_list = store_list[1:]
df = pd.DataFrame(store_list)

In [10]:
df = pd.DataFrame(store_list)

In [11]:
df['url'] = df['url'].str.replace('[^0-9 ]','') #url 숫자만

C:\Users\Heejun\AppData\Local\Temp/ipykernel_3840/2675671744.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['url'] = df['url'].str.replace('[^0-9 ]','') #url 숫자만


In [12]:
df = df.drop_duplicates(['url'])  #중복제거

In [13]:
df.to_csv('마포구치킨매장.csv',encoding='utf-8-sig')

In [14]:
url_list = []

In [15]:
url = 'https://map.naver.com/v5/api/sites/summary/'

In [16]:
for i in df['url']:
    url_list.append(url+i)

In [17]:
params = {
    "lang" : "ko"
}

In [18]:
menu_dict = {'메뉴정보': []}

In [19]:
for i in tqdm(url_list):
    try:
        resp = requests.get(i,params=params).content
        result = json.loads(resp)
        menus = result['menus']
        name = []
        price = []
        x = result['x']
        y = result['y']
        address =  result['address']
        bizhourInfo = result['bizhourInfo']
        for menu in menus:
            name.append(menu.get('name'))
            price.append(menu.get('price'))
        menu_list1 = list(zip(name,price))
        menu_str = ""
        for menu in menu_list1:
            menu_str = menu_str +" "+ "".join(menu)
        menu_list = []
        menu_list.append(menu_str)
        menu_list.append(x)
        menu_list.append(y)
        menu_list.append(address)
        menu_list.append(bizhourInfo)
        menu_dict['메뉴정보'].append(menu_list)
    except:
        menu_dict['메뉴정보'].append("NaN")
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 281/281 [01:04<00:00,  4.37it/s]


In [24]:
for menu in menus:
    name.append(menu.get('name'))
    price.append(menu.get('price'))
menu_list1 = list(zip(name,price))
menu_str = ""
for menu in menu_list1:
    menu_str = menu_str +" "+ "".join(menu)

In [25]:
menu_str

' 순살치킨2마리20,000원 순살치킨3마리25,000원 1가지맛 선택15,000원 감자튀김3,000원 타코야끼5,000원 순살치킨2마리20,000원 순살치킨3마리25,000원 1가지맛 선택15,000원 감자튀김3,000원 타코야끼5,000원 순살치킨2마리20,000원 순살치킨3마리25,000원 1가지맛 선택15,000원 감자튀김3,000원 타코야끼5,000원'

In [26]:
menu_list = []

In [27]:
for i in menu_dict['메뉴정보']:
    menu_list.append(i)

In [28]:
#menu_list = menu_list[:]
df2 = pd.DataFrame(menu_list)

In [29]:
df2

,0,1,2,3,4
0,"오리지날 프린스톤 후라이드치킨18,900원 마틴 간장치킨19,900원 순살 양념치...",126.92189,37.550111,서울 마포구 서교동 409-17,매일 16:00~01:00 포장 16:00 ~ 00:00
1,"골라골라 철인3종세트20,000원 배터져도 책임못짐 철인4종세트26,000원 반반...",126.919811,37.548532,서울 마포구 서교동 402-4 지1층,매일 16:00~02:00
2,"범프오브치킨19,000원 레드핫 칠리 페퍼스19,000원 눈꽃치즈치킨19,000원...",126.929273,37.555217,서울 마포구 서교동 327-22,일요일 17:00~00:00 Last order 23:00 | 토요일 17:00~0...
3,"뿌리노랑 치킨21,000원 3종 치즈볼5,000원 알싸한 마늘치킨 레귤러17,00...",126.922164,37.554073,서울 마포구 서교동 356-3 1층,매일 12:00~01:00 연중무휴
4,"불고기레드그릴21,900원 촉촉칸닭 & 쫄뱅이34,900원 레드그릴20,900원 ...",126.924176,37.556037,서울 마포구 동교동 164-8,"매일 12:00~03:00 금,토요일 04시 까지"
...,...,...,...,...,...
276,N,a,N,None,None
277,"순살 바베큐 치밥9,900원",126.957551,37.547502,서울 마포구 공덕동 118-13,평일 10:00~22:00
278,치킨뱅이변동가격(업주문의) 후라이드치킨변동가격(업주문의) 양념치킨변동가격(업주문의...,126.955301,37.556484,서울 마포구 아현동 327-30,매일 00:00~24:00
279,"베이크치킨18,000원 베이크양념치킨19,000원 베이크(반)+양념(반)19,00...",126.913633,37.554436,서울 마포구 서교동 480-23,평일 11:30~23:00 브레이크타임 : 15:30~17:00 | 토요일 11:3...


In [30]:
df = df.reset_index()

In [31]:
df = df.drop(['index'],axis=1)

In [32]:
df=  pd.concat([df,df2],axis=1)

In [33]:
df

,name,url,0,1,2,3,4
0,아웃닭 홍대점,33758018,"오리지날 프린스톤 후라이드치킨18,900원 마틴 간장치킨19,900원 순살 양념치...",126.92189,37.550111,서울 마포구 서교동 409-17,매일 16:00~01:00 포장 16:00 ~ 00:00
1,철인7호 치킨 홍대점,36803583,"골라골라 철인3종세트20,000원 배터져도 책임못짐 철인4종세트26,000원 반반...",126.919811,37.548532,서울 마포구 서교동 402-4 지1층,매일 16:00~02:00
2,치킨인더키친,20753227,"범프오브치킨19,000원 레드핫 칠리 페퍼스19,000원 눈꽃치즈치킨19,000원...",126.929273,37.555217,서울 마포구 서교동 327-22,일요일 17:00~00:00 Last order 23:00 | 토요일 17:00~0...
3,노랑통닭 홍대점,32588297,"뿌리노랑 치킨21,000원 3종 치즈볼5,000원 알싸한 마늘치킨 레귤러17,00...",126.922164,37.554073,서울 마포구 서교동 356-3 1층,매일 12:00~01:00 연중무휴
4,누구나홀딱반한닭 홍대점,32785743,"불고기레드그릴21,900원 촉촉칸닭 & 쫄뱅이34,900원 레드그릴20,900원 ...",126.924176,37.556037,서울 마포구 동교동 164-8,"매일 12:00~03:00 금,토요일 04시 까지"
...,...,...,...,...,...,...,...
276,하림옛날통닭,1431932616,N,a,N,None,None
277,다마싯 치밥의세계,1098653545,"순살 바베큐 치밥9,900원",126.957551,37.547502,서울 마포구 공덕동 118-13,평일 10:00~22:00
278,마이또치킨뱅이,37684065,치킨뱅이변동가격(업주문의) 후라이드치킨변동가격(업주문의) 양념치킨변동가격(업주문의...,126.955301,37.556484,서울 마포구 아현동 327-30,매일 00:00~24:00
279,핫썬라이스앤치킨,1741030173,"베이크치킨18,000원 베이크양념치킨19,000원 베이크(반)+양념(반)19,00...",126.913633,37.554436,서울 마포구 서교동 480-23,평일 11:30~23:00 브레이크타임 : 15:30~17:00 | 토요일 11:3...


In [34]:
df.to_csv('마포구치킨매장.csv',encoding='utf-8-sig')